# Entrega 2: Análisis Profundo según Enfoque Elegido
- A00836286 | Esteban Sierra Baccio
- A00837527 | Diego de Jesús Esparza Ruíz
- A01722667 | Javier Jorge Hernández Verduzco
- A01285193 | Sergio Omar Flores García
- A01613878 | Sergio Aarón Hernández Orta

## 1. Refinamiento y Operacionalización


### 1.1 Ajustes a las preguntas de investigación basados en feedback de Entrega 1

Tras el feedback, mantuvimos la estructura y el sentido original de las preguntas pero las ajustamos para mayor precisión operativa:

- Se aclaró que *“excluir las que describen la carga”* en la Pregunta 1 significa dejar fuera variable(s) directas de magnitud (Peso total, Volumen, #Remitos) pero mantener transformaciones normalizadas (p. ej. Costo por tonelada) como variables de control.  
- En la Pregunta 2 (estacionalidad) se especificó el nivel temporal a analizar: **día de la semana**, **mes** y **semana móvil**.  
- En la Pregunta 3 (ineficiencias por transportadora/ruta) se añadió un umbral operativo: **rutas con al menos 3 transportistas y ≥ 30 observaciones** para garantizar comparaciones significativas.  
- En la Pregunta 4 se definió qué fuentes externas se considerarán si están disponibles (precio del diesel mensual, tipo de cambio diario y eventos climáticos puntuales) y cómo se integrarán (join por fecha/mes).  
- En la Pregunta 5 se estableció la métrica exacta de desviación: **Desviación relativa = (Costo real − Costo óptimo simulado) / Costo óptimo simulado**, y se definió que la proporción de varianza atribuible a estas desviaciones será la **varianza(Desviación) / varianza(Costo real)**.

Estos ajustes responden a comentarios sobre reproducibilidad y claridad en las métricas a reportar.

---

### 1.2 Definición operacional precisa de cada variable y constructo

Se listan las variables principales (observadas y derivadas) con su definición, tipo y tratamiento inicial.

#### Variables dependientes
- **Costo (MXN)** — *numérica continua*. Costo total registrado por viaje en pesos mexicanos. Uso directo; revisar outliers (valores ≫ P99) y truncar/ winsorizar si procedente.
- **CostoxTn** — *numérica continua*. `Costo / (Peso total (kg) / 1000)`. Se usa como normalización por tonelada.

#### Variables independientes (selección)
- **Transp.Leg / Nombre** — *categórica nominal*. Identificador de transportista. Codificación: dummies o efecto aleatorio según especificación.
- **Ori-Dest** — *categórica nominal*. Par origen-destino compreso como string; también se desagrega en `Origen` y `Destino`.
- **Tipo de planta / Tipo Planta** — *categórica ordinal/nominal* (categorías: masivos, revestidos, customizados). Dummies.
- **F.Salida, H.Salida** — *fecha / hora*. Derivados: `Mes`, `DíaSemana`, `HoraBloque` (p. ej. 0–6, 6–12, ...), `Temporada` (Q1..Q4).
- **Peso total (kg)** — *numérica continua*. Control de magnitud.
- **Flete Falso (MXN)** — *numérica continua*. Se tratará como imputable o indicador de corrección; posible outlier.

#### Variables con tratamiento especial
- **TpoVje** — *categórica / temporal* (2,847 missings). Se crea una categoría `Desconocido` y flag `TpoVje_missing = 1` para preservar información faltante.
- **Costo Prom, Variación** — (1,121 missings). Se reconstruirán conceptualmente y se comparará con columna original; si no es recuperable, se imputará por mediana condicionada a `Ori-Dest` y `Transportista`.

---

### 1.3 Justificación de cualquier cambio en el enfoque metodológico

El feedback confirmó que el **enfoque primario econométrico** es apropiado. Confirmamos ese foco por las razones siguientes:

- Los datos proporcionados son ricos en atributos internos (rutas, transportistas, fechas) pero carecen de cobertura exhaustiva de variables externas de mercado. Para **identificar y cuantificar efectos** controlando por covariables observables, modelos econométricos (MLR con efectos fijos/mixtos) son más interpretables y útiles para la toma de decisiones empresariales inmediatas.
- Mantendremos un componente de **optimización** secundario (simulación de costo óptimo) para convertir hallazgos en métricas económicas accionables.
- No se priorizó como método principal un pipeline de ML “black-box” porque, sin datos externos y con la necesidad de interpretabilidad para el socio, el valor adicional sería menor. No obstante, se evaluarán modelos de boosting y regularizados como comparación predictiva y para identificar variables importantes.

---

### 1.4 Descripción detallada de cómo construyeron variables derivadas o índices

A continuación se describen las variables derivadas y los índices que ya se construyeron o se planifican construir, con su **fórmula**, **propósito** y **tratamiento de valores faltantes**.

1. **Costo por tonelada (CostoxTn)**  
   - **Fórmula:** `CostoxTn = Costo / (Peso total (kg) / 1000)`  
   - **Propósito:** Normalizar el costo por tamaño de la carga para comparar viajes heterogéneos.  
   - **Tratamiento missing/outliers:** Si `Peso total` = 0 o missing → marcar `invalid_weight=1` y excluir de análisis de CostoxTn; imputar peso por mediana de `Ori-Dest` si procedente.

2. **Índice de aprovechamiento de carga (LoadUtilIndex)**  
   - **Fórmula conceptual:** `Peso total / (Peso total + Flete Falso (kg))` (bounded 0–1).  
   - **Propósito:** Medir la eficiencia en el uso de capacidad del vehículo; valores bajos indican subutilización.  
   - **Tratamiento:** Si `Flete Falso (kg)` missing → imputar 0 si no hay evidencia de aplicación; crear flag `FF_missing`.

3. **Desviación de costo teórico (CostDeviation)**  
   - **Fórmula:** `Desviación = (Costo real - Costo óptimo simulado) / Costo óptimo simulado`  
   - **Propósito:** Cuantificar la prima pagada respecto a una asignación hipotética óptima (usando la transportista más barata observada bajo condiciones similares).  
   - **Construcción:** Para cada viaje se simula el “Costo óptimo” usando el modelo predictivo (controlando Ori-Dest, peso, tipo de servicio) o asignando el costo medio por `Ori-Dest` y `TipoServicio` del transportista más barato.  
   - **Tratamiento:** Si no hay comparables (pocas observaciones), excluir de la estadística de proporción global pero reportar casos.

4. **Elasticidad aproximada del costo (CostElasticity_X)**  
   - **Fórmula:** `(ΔCosto / Costo) / (ΔX / X)` para variables continuas X (ej. Peso).  
   - **Propósito:** Medir sensibilidad porcentual del costo ante cambios unitarios en X. Se estimará a partir de coeficientes estandarizados de regresión log-log cuando aplique.  
   - **Tratamiento:** Log-transformaciones sólo cuando variable > 0; missing → exclusión puntual.

5. **Índice de competencia por ruta (RouteCompetitionIndex)**  
   - **Fórmula (operativa):** `1 − (Herfindahl-Hirschman Index de participación por # viajes en la ruta)`  
   - **Propósito:** Señalar rutas con alta concentración de mercado (índice bajo = alta competencia).  
   - **Construcción:** Para cada `Ori-Dest` calcular shares por transportista y derivar HHI.  
   - **Tratamiento:** Rutas con < 3 transportistas o < 30 observaciones → marcar `insufficient_competition_data`.

6. **Flags y dummies temporales**  
   - **Variables:** `Mes`, `DíaSemana`, `SemanaMovilAvgCost` (rolling 4 semanas).  
   - **Propósito:** Capturar estacionalidad y estructura temporal no lineal.  
   - **Tratamiento:** Missing en fecha → excluir; `TpoVje_missing` creado como dummy para preservar señal.

7. **Indicador de transportista caro (HighCostCarrierFlag)**  
   - **Fórmula:** transportista cuyo costo promedio en una ruta > promedio ruta + 1.5 * SD.  
   - **Propósito:** Detectar transportistas que consistentemente cobran primas.  
   - **Uso:** Variable categórica en análisis de ineficiencia; sensibilidad a outliers mitigada por usar medianas.

8. **Variables de imputación y flags de calidad**  
   - Para variables con muchos missings (TpoVje, Costo Prom, Variación) se crea un **flag** (`*_missing`) para cada variable imputada y se documenta la regla de imputación (mediana por grupo, categoría "Desconocido", o exclusión). Estos flags se usan en modelos para capturar información contenida en la ausencia.

---

 

## 2. Implementación Metodológica Principal (5-7 páginas + código)

El contenido específico dependerá del enfoque elegido, pero TODOS deben incluir:

### Sección A: Preparación y Transformación

#### Proceso detallado de limpieza y transformación de datos
| Variable a eliminar | Razón |
| ---- | ---- |
| Viaje | Información irrelevante |
| Permiso | Información irrelevante |
| Shipment | Información irrelevante |
| Sociedad | Información irrelevante |
| Ori-Dest-TT | Información redundante |
| Ori-Dest | Información redundante |
| Modal | Información irrelevante |
| Moneda | Información irrelevante |
| Estatus | Información irrelevante |


In [ ]:
# Leer los datos
import pandas as pd
import numpy as np

df = pd.read_excel("../data/raw/Viajes Sep-Dic 24 v2.xlsx", sheet_name='Viajes')

### Feature: Distancia recorrida en KM

In [ ]:
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import time

def calcular_distancia_recta(row):
    try:
        geolocator = Nominatim(user_agent="mi_app_geo") # Define tu user_agent
        
        # Concatena tus campos para darle más contexto a Nominatim
        query_origen = f"{row['Desc Org. Apt']}, {row['Origen']}"
        query_destino = f"{row['Destino']}"

        # Geocodifica el origen
        location_origen = geolocator.geocode(query_origen)
        time.sleep(1) # IMPORTANTE: Nominatim requiere un delay de 1 seg por petición
        
        # Geocodifica el destino
        location_destino = geolocator.geocode(query_destino)
        time.sleep(1)

        if location_origen and location_destino:
            coords_origen = (location_origen.latitude, location_origen.longitude)
            coords_destino = (location_destino.latitude, location_destino.longitude)
            
            # Calcula la distancia geodésica (línea recta)
            distancia_km = geodesic(coords_origen, coords_destino).kilometers
            print(f"Éxito: {query_origen} -> {query_destino} = {distancia_km:.2f} km")
            return distancia_km
        else:
            print(f"No se pudieron geocodificar: {query_origen} o {query_destino}")
            return np.nan

    except Exception as e:
        print(f"Error en Nominatim: {e}")
        return np.nan

In [ ]:
# Modificamos el DF para estandarizar nombres de Origen y Destino
df['Desc Org. Apt'] = df['Origen'].replace('Churubusco', 'Churubusco Monterrey')
df['Destino'] = df['Destino'].replace('AM Monterrey Plantas', 'Monterrey')
df['Destino'] = df['Destino'].replace('AM Monterrey Local', 'Monterrey')
df['Destino'] = df['Destino'].replace('Area Metr Queretaro', 'Queretaro')
df['Destino'] = df['Destino'].replace('Área Metro Saltillo', 'Saltillo')
df['Destino'] = df['Destino'].replace('Area Metro Monclova', 'Monclova')
df['Destino'] = df['Destino'].replace('Área Metro Monclova', 'Monclova')
df['Destino'] = df['Destino'].replace('Área Metro SLP', 'San Luis Potosi')
df['Destino'] = df['Destino'].replace('Área Metro Puebla', 'Puebla')
df['Destino'] = df['Destino'].replace('Área Metro Celaya', 'Celaya')
df['Destino'] = df['Destino'].replace('A. M. Aguascalientes', 'Aguascalientes')

# Creamos un dataframe con las combinaciones únicas de Origen y Destino
combinaciones = df[['Desc Org. Apt','Origen', 'Destino']].drop_duplicates()

# Obtenemos la distancia entre cada par de Origen y Destino
combinaciones['Distancia_km'] = combinaciones.apply(calcular_distancia_recta, axis=1)

# Añadimos la distancia al dataframe original
df = df.merge(combinaciones[['Desc Org. Apt','Origen', 'Destino', 'Distancia_km']], on=['Desc Org. Apt','Origen', 'Destino'], how='left')

# Guardamos las combinaciones en un archivo CSV
combinaciones.to_csv('../data/processed/combinaciones_origen_destino.csv', index=False)

#print(combinaciones)

### Features: Tipo Planta

In [ ]:
# One Hot Encoding para la columna 'Tipo Planta'
tipo_planta_dummies = pd.get_dummies(df['Tipo Planta'], prefix='Tipo_Planta')
df = pd.concat([df, tipo_planta_dummies], axis=1)

# Eliminar variable original
df.drop('Tipo Planta', axis=1, inplace=True)

### Features: Depósito Origen

In [ ]:
# one hot encoding para la columna 'Depósito Origen'
deposito_origen_dummies = pd.get_dummies(df['Deposito Origen'], prefix='Deposito_Origen')
df = pd.concat([df, deposito_origen_dummies], axis=1)

# Eliminar variable original
df.drop('Deposito Origen', axis=1, inplace=True)

### Features: Tipo de permiso

In [ ]:
# One hot encoding para la columna 'Tipo de permiso'
tipo_permiso_dummies = pd.get_dummies(df['TpoPermiso'], prefix='Tipo_Permiso')
df = pd.concat([df, tipo_permiso_dummies], axis=1)

# Eliminar variable original
df.drop('TpoPermiso', axis=1, inplace=True)

## Features: Tipo de Servicio

In [ ]:
# One hot encoding para la columna 'Tipo Servicio'
tipo_servicio_dummies = pd.get_dummies(df['TpoSrv'], prefix='Tipo_Servicio')
df = pd.concat([df, tipo_servicio_dummies], axis=1)

# Eliminar variable original
df.drop('TpoSrv', axis=1, inplace=True)

## Features: Tipo de camion

In [ ]:
# Feature: 'Ejes'
def obtener_ejes(tipo_camion):
    if pd.isnull(tipo_camion):
        return 0
    elif tipo_camion == 'Big Coil':
        return 3
    elif tipo_camion == '3 Ejes':
        return 3
    elif tipo_camion == '3 Eje Cortina':
        return 3
    elif tipo_camion == '3 ejes Portacinta':
        return 3
    elif tipo_camion == "Plataforma 48'" or tipo_camion == "Plataforma 53'":
        return 4
    elif tipo_camion == "2 Ejes":
        return 2

df['Ejes'] = df['Tipo transporte'].apply(obtener_ejes)

### Eliminacion de datos

In [ ]:
# Limpieza de datos
df = df.drop(columns=['Viaje', 'Permiso', 'Shipment', 'Sociedad','Ori-Dest-TT','Ori-Dest','Modal.','Moneda','Estatus'])

df = df.drop(columns=['Planta Origen', 'Origen', 'Destino', 'Desc Org. Apt', 'TPP', 'Desc TPP', 'Transp.Leg'])

# Se elimina la última linea que contiene totales
df = df.iloc[:-1]

df.head()


In [ ]:
df.head()


#### Justificación de decisiones sobre valores faltantes, outliers, y agregaciones
#### Creación de variables relevantes para su enfoque (ratios, interacciones, lags, etc.)
#### Validación de supuestos requeridos por sus métodos
 

⠀Sección B: Análisis Principal (varía según enfoque)

Si eligieron enfoque econométrico:

Especificación y estimación de modelos (efectos fijos, aleatorios, mixtos)
Tests de especificación (Hausman, Breusch-Pagan, etc.)
Análisis de elasticidades y efectos marginales
Descomposición de varianza (Oaxaca-Blinder, Shapley, etc.)
 

⠀Si eligieron optimización:

Formulación matemática del problema de optimización
Implementación y solución del modelo
Análisis de sensibilidad y precios sombra
Comparación entre solución óptima y realidad observada
Identificación de restricciones activas
 

⠀Si eligieron machine learning:

Feature engineering creativo y justificado
Comparación sistemática de algoritmos con justificación
Análisis de importancia de variables (SHAP, permutación, etc.)
Interpretación detallada del modelo seleccionado
Diagnóstico de sesgo-varianza
 

⠀Si eligieron análisis causal:

Construcción y justificación del DAG (Directed Acyclic Graph)
Identificación de confounders, mediadores, y colisionadores
Estrategia de identificación causal (IV, diferencias-en-diferencias, etc.)
Estimación de efectos causales con intervalos de confianza
Análisis de sensibilidad a violaciones de supuestos
 

⠀Si eligieron sistemas complejos:

Modelado de interacciones y retroalimentación
Identificación de puntos de inflexión o transiciones de fase
Análisis de propagación de efectos
Simulación de escenarios emergentes

## 3. Validación Rigurosa (2-3 páginas)

Validación cruzada temporal (no aleatoria - respeten la estructura temporal)
Tests de robustez: ¿Los resultados se mantienen bajo diferentes especificaciones?
Análisis de estabilidad temporal: ¿Las relaciones encontradas son consistentes?
Bootstrap o métodos de remuestreo para cuantificar incertidumbre
Diagnósticos específicos del método (multicolinealidad, heterocedasticidad, autocorrelación, etc.)
 

## 4. Análisis de Casos Especiales (2 páginas)

Investigación detallada de outliers o períodos anómalos
¿Qué revelan estos casos sobre su teoría?
¿Son errores, casos especiales, o evidencia contra sus hipótesis?
Análisis de subgrupos relevantes (por tipo de transportista, ruta, temporada, etc.)
 

## 5. Primeros Hallazgos y Respuestas Tentativas (3-4 páginas)

Para cada pregunta de investigación:

Evidencia empírica encontrada (con visualizaciones apropiadas)
Magnitud y significancia de los efectos (práctica, no solo estadística)
Interpretación en el contexto del negocio
Nivel de confianza en el hallazgo
Explicaciones alternativas que no pueden descartarse
 

⠀6. Reflexión Crítica sobre Limitaciones (1-2 páginas)

Supuestos que no pudieron validar completamente
Limitaciones de los datos para responder sus preguntas
Amenazas a la validez interna y externa
Sesgos potenciales en su análisis
Lo que su método NO puede revelar sobre el problema